<a href="https://colab.research.google.com/github/Dykim991222/MLstudy/blob/main/%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain==0.3.0 langchain-core langchain-huggingface langchain-community unsloth

  Using cached langchain_core-1.2.5-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_huggingface-1.2.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_core-0.3.81-py3-none-any.whl.metadata (3.2 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.80-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.78-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.77-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.76-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_core-0.3.75-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_core-0.3.74-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is still looking at multiple versions 

In [ ]:
# Chain 생성
# chain = prompt | model | output_parser

from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.llms import HuggingFacePipeline
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# 모델 로드( Unsloth )

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Mistral-7B-Instruct-v0.3',
    max_seq_length=2048, # 질문 + 컨텍스트 + 답변 을 모두 합친 "전체토큰수"의 최대치
    dtype=None,
    load_in_4bit=True, # 4-bit 양자화. 숫자를 아주작은 정밀도로 "압축저장"
)
model.eval

==((====))==  Unsloth 2025.12.9: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

<bound method PreTrainedModel.eval of MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096, padding_idx=770)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-05)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4096, out_features=32768, bias=False)
)>

In [ ]:
# HuggingFacePipeline 래핑
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256, #출력 전용 제한
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
/tmp/ipython-input-1890511675.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
# PromptTemplate 정의
prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
        "You are a helpful assistant that always responds in Korean.\n"
        "Answer politely and concisely.\n\n"
        "### User input :\n{user_input}\n\n"
        "### Assistant response (answer just once):"
    ),
)

In [ ]:
# Output Parser 정의
output_parser = StrOutputParser()

# Chain 구성
chain = prompt | llm | output_parser

# 실행
result = chain.invoke({"user_input": "My name is DoyeonKim"})
print(result)

You are a helpful assistant that always responds in Korean.
Answer politely and concisely.

### User input :
My name is DoyeonKim

### Assistant response (answer just once):
Nice to meet you, DoyeonKim. It's a pleasure to assist you. What can I help you with today?


# 출력 파서 (Parser)

LLM이 생성한 "텍스트 출력"을 사람이/프로그램이 쓰기 좋은 "구조화된 형태"로 변환하는 역할

In [ ]:
from langchain.schema import StrOutputParser

output_parser = StrOutputParser()

prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
              "You are a helpful assistant that always responds in Korean.\n"
              "Output *only* valid JSON and nothing else.\n"
              "Do not include any explanations, text, or comments outside of JSON.\n\n"
              "{user_input}\n\n"
              "Respond strictly in JSON format:"
    ),
)


chain = prompt | llm | output_parser

# 실행
result = chain.invoke({"user_input": "일주일치 식단 줘"})
print(result)

You are a helpful assistant that always responds in Korean.
Output *only* valid JSON and nothing else.
Do not include any explanations, text, or comments outside of JSON.

일주일치 식단 줘

Respond strictly in JSON format:

```json
{
  "weekly_diet": [
    {
      "day": "월",
      "breakfast": "아침: 빵 2조, 치즈 1조, 생선 1조",
      "lunch": "점심: 돼지고기 1조, 콩나물 1조, 칼로리 100kcal",
      "dinner": "저녁: 멍고기 1조, 양파 1조, 칼로리 200kcal"
    },
    {
      "day": "화",
      "breakfast": "아침: 빵 2조, 치즈 1조, 생선 1조",
      "lunch": "점심: 소고기 1조, 콩나물 1조, 칼로리 120kcal",
      "dinner": "저녁: 멍고기 1조, 양파 1조, 칼로리 250kcal"
    },
    {
      "day": "수",
      "breakfast": "아침: 빵 2조, 치즈 1조, 생선 1조",
      "lunch": "점심: 돼지고기 1조, 콩나물 1조, 칼로리 150kcal",
      "dinner": "저녁: 멍고기 1조, 양파 1조, 칼로리 220kcal"
    },
    {
      "day": "목",
      "breakfast": "아침: 빵 2조, 치즈 1조, 생선 1조",
      "lunch": "점심: 소고기 1조, 콩나물 1조, 칼로리 130kcal",
      "dinner": "저녁: 멍고기 1조, 양파 1조, 칼로리 230kcal"
    },
    {
      "day": "금",
      "breakfast": "아침: 빵 2조

In [ ]:
type(result)

str

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# 모델을 JSON으로 출력하는 도구가 아닌 JSON형태로 나온 문장을 dict로 변환해주는 도구.

output_parser = JsonOutputParser()

output_parser = JsonOutputParser()

prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
              "You are a helpful assistant that always responds in Korean.\n"
              "Output *only* valid JSON and nothing else.\n"
              "Do not include any explanations, text, or comments outside of JSON.\n\n"
              "{user_input}\n\n"
              "Respond strictly in JSON format:"
    ),
)

chain = prompt | llm | output_parser

# 실행
result = chain.invoke({"user_input": "일주일치 식단 줘"})
print(result)

{'weekly_diet': [{'day': '월', 'breakfast': '콩나물 국', 'lunch': '돈까스', 'dinner': '삼계탕', 'snack': '오렌지'}, {'day': '화', 'breakfast': '김치 전', 'lunch': '김치 찌개', 'dinner': '닭가슴살', 'snack': '사과'}, {'day': '수', 'breakfast': '딸기 야채찹찮', 'lunch': '김치 전', 'dinner': '삼계탕', 'snack': '바나나'}, {'day': '목', 'breakfast': '빵 빵이', 'lunch': '삼겹살', 'dinner': '돈까스', 'snack': '포도'}, {'day': '금', 'breakfast': '토마토 아침구슬', 'lunch': '김치 찌개', 'dinner': '삼계탕', 'snack': '오렌지'}, {'day': '토', 'breakfast': '딸기 야채찹찮', 'lunch': '닭가슴살', 'dinner': '삼계탕', 'snack': '사과'}, {'day': '일', 'breakfast': '콩나물 국', 'lunch': '돈까스', 'dinner': '삼계탕', 'snack': '오렌지'}]}


In [ ]:
type(result)

dict

# LLM 모델 파인튜닝
Unsloth : LLM을 파인튜닝 및 추론을 지원하는 라이브러리

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

# 4비트 사전 양자화된 모델 리스트 (더 빠른 다운로드 및 OOM 방지)

    # "unsloth/Meta-Llama-3.1-8B-bnb-4bit"    # Llama-3.1 8B 모델, 4비트 양자화
    # "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    # "unsloth/Meta-Llama-3.1-70B-bnb-4bit"
    # "unsloth/Meta-Llama-3.1-405B-bnb-4bit"   # 405B 모델도 4비트 지원
    # "unsloth/Mistral-Nemo-Base-2407-bnb-4bit" # Mistral 12B 모델, 4비트 지원
    # "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit"
    # "unsloth/mistral-7b-v0.3-bnb-4bit"        # Mistral v3 7B 모델, 4비트 지원
    # "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
    # "unsloth/Phi-3.5-mini-instruct"         # Phi-3.5 미니 인스트럭트 모델
    # "unsloth/Phi-3-medium-4k-instruct"
    # "unsloth/gemma-2-9b-bnb-4bit"
    # "unsloth/gemma-2-27b-bnb-4bit"           # Gemma 2.27B 모델, 4비트 지원
# 더 많은 모델은 https://huggingface.co/unsloth 에서 확인 가능

# 사전 학습된 모델과 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "LGAI-EXAONE/EXAONE-4.0-1.2B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.12.9: Fast Exaone4 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
# PEFT(Parameter Efficient Fine-tuning)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA 랭크 설정. r이 클수록 더많은 정보 학습, 너무크면 메모리 잡아먹음
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"], # PEFT 적용할 모듈 목록. 모델의 특정 부분(모듈)에만 학습
    lora_alpha = 16, # LoRA 알파 설정. LoRA라는 기술 얼마나 강하게 작용할지 조절
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False, # 랭크 안정화 LoRA 사용여부
    loftq_config = None, # LoftQ 설정
)

In [ ]:
# 모델에게 주어질 텍스트의 형식을 정의
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

# 맨윗줄: 모델에게 앞으로 제공될 지시사항을 기반으로 적절한 응답을 작성하라고 말함.
# 모델이 어떤 작업을 수행해야 하는지 명확히 이해할 수 있도록 도와줌
# Instruction:은 지시사항이 제공될 부분
# Response:는 모델이 생성해야 할 응답이 제공될 부분

# EOS 토큰 가져오기 (생성 종료를 위해 필요)
EOS_TOKEN = tokenizer.eos_token  # 반드시 EOS_TOKEN을 추가해야 함
EOS_TOKEN

'[|endofturn|]'

In [ ]:
# 프롬프트 포맷팅 함수 정의
def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 데이터셋의 'instruction' 필드
    outputs      = examples["output"]       # 데이터셋의 'output' 필드
    texts = []
    for instruction, output in zip(instructions, outputs):
                                                           # EOS_TOKEN을 추가하지 않으면 생성이 무한히 계속됨
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN  # 프롬프트 형식에 맞게 텍스트 생성
        texts.append(text)
    return { "text" : texts, }  # 'text' 필드로 반환

import pandas as pd
from datasets import load_dataset, Dataset,concatenate_datasets  # Hugging Face datasets 라이브러리 임포트

# 데이터셋 로드 (Teddy Lee의 QA 데이터셋 미니 버전)
dataset = load_dataset("teddylee777/QA-Dataset-mini", split = "train")

new_data = [
    {"instruction": "성수에 있는 인공지능 학원은 어디입니까?", "output": "알파코캠퍼스 입니다"}
]

# 새로운 데이터셋 생성 (기존 데이터 + 새로운 데이터)
new_dataset = Dataset.from_pandas(pd.DataFrame(new_data))

# 기존 데이터셋에 새로운 데이터 추가
dataset = concatenate_datasets([dataset, new_dataset])

# 프롬프트 포맷팅 함수 적용하여 데이터셋 변환
dataset = dataset.map(formatting_prompts_func, batched = True,)


# 예제 엑셀 형태로 데이터셋을 만든다면?
# from datasets import load_dataset
# dataset = load_dataset( "csv", data_files = "data.csv", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched=True)
#

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/6.20k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [ ]:
print("데이터셋 확인")
for i, item in enumerate(dataset["text"][-5:]):  # 마지막 5개만 보기
    print(f"\n[{i+1}] --------------------")
    print(item)

데이터셋 확인

[1] --------------------
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
CES 2024 전시회의 주제는 무엇이며, 어떤 산업을 포괄한다고 언급되었나요?

### Response:
CES 2024 전시회의 주제는 '올 온(All on)'이며, 모든 산업을 포괄한다고 언급되었습니다.[|endofturn|]

[2] --------------------
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
테디노트 유튜브 채널에 대해서 알려주세요.

### Response:
테디노트(TeddyNote)는 데이터 분석, 머신러닝, 딥러닝 등의 주제를 다루는 유튜브 채널입니다. 이 채널을 운영하는 이경록님은 데이터 분석과 인공지능에 대한 다양한 강의를 제공하며, 초보자도 쉽게 따라할 수 있도록 친절하게 설명합니다.[|endofturn|]

[3] --------------------
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
랭체인 관련 튜토리얼은 어디서 찾을 수 있나요?

### Response:
테디노트의 위키독스 페이지에는 LangChain에 대한 다양한 한국어 튜토리얼이 제공됩니다. 링크: https://wikidocs.net/book/14314[|endofturn|]

[4] --------------------
Below is an instruction that describes a task. Write a r